In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

# Dataset paths
dataset_paths = {
    1000: "Data_1000.csv",
    10000: "Data_10000.csv",
    100000: "Data_100000.csv"
}

results = []

# Loop over datasets and configurations
for size, path in dataset_paths.items():
    print(f"\nProcessing dataset size: {size}")
    df = pd.read_csv(path)
    X = df.drop(columns=["outcome"]).values
    y = df["outcome"].values

    # Train-validation split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Feature scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    for config in ["1-layer", "2-layer"]:
        print(f"  Training {config} model...")
        model = models.Sequential()
        model.add(layers.Input(shape=(X.shape[1],)))
        model.add(layers.Dense(4, activation='relu'))
        if config == "2-layer":
            model.add(layers.Dense(4, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Measure execution time
        start_time = time.time()
        history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_val, y_val))
        end_time = time.time()

        train_acc = history.history['accuracy'][-1]
        val_acc = history.history['val_accuracy'][-1]

        results.append({
            "Dataset Size": size,
            "Model Config": config,
            "Training Error": round(1 - train_acc, 4),
            "Validation Error": round(1 - val_acc, 4),
            "Time Taken (s)": round(end_time - start_time, 2)
        })

# Output results
results_df = pd.DataFrame(results)
print("\nSummary Table:")
print(results_df)

# Optionally save to CSV
results_df.to_csv("deep_learning_results.csv", index=False)



Processing dataset size: 1000
  Training 1-layer model...
  Training 2-layer model...

Processing dataset size: 10000
  Training 1-layer model...
  Training 2-layer model...

Processing dataset size: 100000
  Training 1-layer model...
  Training 2-layer model...

Summary Table:
   Dataset Size Model Config  Training Error  Validation Error  Time Taken (s)
0          1000      1-layer          0.0950            0.0900            6.49
1          1000      2-layer          0.0612            0.0700            8.18
2         10000      1-layer          0.0034            0.0005           26.01
3         10000      2-layer          0.0038            0.0025           25.50
4        100000      1-layer          0.0014            0.0010          176.49
5        100000      2-layer          0.0018            0.0018          193.81
